In [1]:
import torch
import random
import os

from torch.nn.utils.rnn import pad_sequence
from datasets import load_dataset
from torch.utils.data import DataLoader
from functools import partial
from tqdm import tqdm
from dataclasses import dataclass
from peft import LoraConfig, get_peft_model
from transformers import RobertaTokenizer, T5ForConditionalGeneration

In [2]:
PATH = os.getcwd()
SEED = 42
LR = 2e-5
BATCH_SIZE = 8
EPOCHS = 10

In [4]:
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.enabled = True


In [5]:
set_seed(SEED)
print(f"Seed: {SEED}")

Seed: 42


In [6]:
tokenizer = RobertaTokenizer.from_pretrained('Salesforce/codet5-small')
model = T5ForConditionalGeneration.from_pretrained('Salesforce/codet5-small')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/242M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

In [7]:
mbpp = load_dataset("mbpp", "sanitized")

README.md: 0.00B [00:00, ?B/s]

sanitized/train-00000-of-00001.parquet:   0%|          | 0.00/33.9k [00:00<?, ?B/s]

sanitized/test-00000-of-00001.parquet:   0%|          | 0.00/60.9k [00:00<?, ?B/s]

sanitized/validation-00000-of-00001.parq(…):   0%|          | 0.00/14.0k [00:00<?, ?B/s]

sanitized/prompt-00000-of-00001.parquet:   0%|          | 0.00/6.72k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/257 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/43 [00:00<?, ? examples/s]

Generating prompt split:   0%|          | 0/7 [00:00<?, ? examples/s]

In [8]:
mbpp['train'][0]

{'source_file': 'Benchmark Questions Verification V2.ipynb',
 'task_id': 602,
 'prompt': 'Write a python function to find the first repeated character in a given string.',
 'code': 'def first_repeated_char(str1):\n  for index,c in enumerate(str1):\n    if str1[:index+1].count(c) > 1:\n      return c',
 'test_imports': [],
 'test_list': ['assert first_repeated_char("abcabc") == "a"',
  'assert first_repeated_char("abc") == None',
  'assert first_repeated_char("123123") == "1"']}